In [1]:
import $file.common
import common._, dc._
import doobie.implicits._

import $file.$     

import common._, dc._

import doobie.implicits._

# Variation 7. The good: Quill & LINQ DSLs

The data model is defined by case classes.

In [2]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    population: Int)

defined class Country
defined class City

Queries use Scala syntax and build upon case classes:

In [3]:
val largeCapitals = quote { for {
    country <- query[Country] 
    city <- query[City] if country.capital.exists(_ == city.id) 
    if city.population > 8000000
} yield (country.name, city.name) }

largeCapitals: AnyRef with Quoted[Query[(String, String)]]{def quoted: io.getquill.ast.FlatMap;def ast: io.getquill.ast.FlatMap;def id1759979198(): Unit;val liftings: Object} = querySchema("Country").flatMap(country => querySchema("City").filter(city => country.capital.exists((x1) => x1 == city.id)).filter(city => city.population > 8000000).map(city => (country.name, city.name)))

But they are truly generic. It can be translated into an SQL query:

In [4]:
run(largeCapitals).transact(xa).unsafeRunSync.timed

cmd3.sc:1: SELECT country.name, city.name FROM Country country, City city WHERE country.capital = city.id AND city.population > 8000000
val res3 = run(largeCapitals).transact(xa).unsafeRunSync.timed
              ^449 millis


res3: List[(String, String)] = List(
  ("Indonesia", "Jakarta"),
  ("South Korea", "Seoul"),
  ("Mexico", "Ciudad de M\u00e9xico"),
  ("Russian Federation", "Moscow")
)

or into Cassandra (TBD)